In [6]:
import pickle
import folium
import requests
import numpy as np
import random
from pyproj import Proj
from eviltransform import gcj2wgs
lon_cen, lat_cen = 116.4213, 39.9636
WGS = 0
GCJ = 1
BD = 2
projector = Proj("+proj=utm +zone=50 +ellps=WGS84 +datum=WGS84 +units=m +no_defs")

In [3]:
regions = pickle.load(open("data/regions.pkl", "rb"))

In [11]:
G_folium = folium.Map(
    location=[lat_cen, lon_cen],
    control_scale=True
)
for r in regions:
    folium.PolyLine(
        locations=[p[::-1] for p in r["boundary"]],
        opacity=0.8,
        weight=3,
        popup=r["name"],
    ).add_to(G_folium)
G_folium.save("figure/regions.html")

In [18]:
## 高德地图API
user_key = "7b111df4c999cad85c134530eea33e0e"
# keywords = "展春园小区"
# keywords = "北京朝阳区三环以内北京市朝阳区和平街12区甲19号楼"
keywords = "北京朝阳区三环以内和平街12区13号楼1单元8号"
type = 190000  # 高德POI分类码, 190000为地名地址信息, 即xx小区xx号楼这种
citycode = 110000
adcode = 110105


# 关键字搜索(结果不准)
url_keywords_search = f"https://restapi.amap.com/v5/place/text?key={user_key}&keywords={keywords}&types={type}&city=北京&children=1&offset=20&page=1&extensions=all"
# url_keywords_search = "https://restapi.amap.com/v5/place/text?key=7b111df4c999cad85c134530eea33e0e&keywords=北京朝阳区三环以内和平街12区13号楼1单元8号&region=北京市&types=190000"

# 地理编码(效果还行,但只能获得一个经纬度)
url_geo_code = f"https://restapi.amap.com/v3/geocode/geo?key={user_key}&address={keywords}&types={type}&region=北京"
# url_geo_code = "https://restapi.amap.com/v3/geocode/geo?key=7b111df4c999cad85c134530eea33e0e&address=北京朝阳区三环以内北京市朝阳区和平街12区甲19号楼&city=110000"

t = requests.get(url_geo_code, headers={"User-Agent": random.choice(user_agents)}).json()
print(t)

address = t["geocodes"][0]["formatted_address"]
lonlat = t["geocodes"][0]["location"]
print(address, lonlat)

# 使用经纬度进行 周边搜索
url_near_search = f"https://restapi.amap.com/v3/place/around?key={user_key}&location={lonlat}&keywords=&types={type}&radius=100&offset=20&page=1&extensions=all"
# url_near_search = "https://restapi.amap.com/v3/place/around?key=7b111df4c999cad85c134530eea33e0e&location=116.428283,39.968908&types=190000&radius=50&offset=20&page=1&extensions=all"
print(url_near_search)
t = requests.get(url_near_search, headers={"User-Agent": random.choice(user_agents)}).json()
print(t)

{'status': '1', 'info': 'OK', 'infocode': '10000', 'count': '1', 'geocodes': [{'formatted_address': '北京市朝阳区和平街12区|13号楼', 'country': '中国', 'province': '北京市', 'citycode': '010', 'city': '北京市', 'district': '朝阳区', 'township': [], 'neighborhood': {'name': [], 'type': []}, 'building': {'name': [], 'type': []}, 'adcode': '110105', 'street': [], 'number': [], 'location': '116.425940,39.966468', 'level': '门牌号'}]}
北京市朝阳区和平街12区|13号楼 116.425940,39.966468
https://restapi.amap.com/v3/place/around?key=7b111df4c999cad85c134530eea33e0e&location=116.425940,39.966468&keywords=&types=190000&radius=100&offset=20&page=1&extensions=all
{'suggestion': {'keywords': [], 'cities': []}, 'count': '0', 'infocode': '10000', 'pois': [], 'status': '1', 'info': 'OK'}


In [4]:
# 获取AOI边界接口(有滑块验证)
url='https://ditu.amap.com/detail/get/detail?id=B000A9NK9G'

In [41]:
# buildings
bds = [
    ["和平街十二区20号楼", [(116.427821,39.96892), (116.42869,39.96894), (116.428701,39.968829), (116.427816,39.968813)]],
    ["和平街十二区19号楼", [(116.426276,39.969039), (116.427666,39.969072), (116.427666,39.968969), (116.426276,39.968932)]],
    ["和平街十二区18号楼", [(116.426282,39.968562), (116.426282,39.968735), (116.426346,39.968735), (116.426357,39.968813), (116.426904,39.968821), (116.426904,39.968722), (116.4264,39.968702), (116.426389,39.96857)]],
    ["和平街十二区17号楼", [(116.425761,39.968545), (116.426303,39.968566), (116.426314,39.968439), (116.425767,39.968418)]],
    ["和平街十二区11号楼", [(116.425203,39.968562), (116.425622,39.968566), (116.425633,39.968443), (116.425209,39.968439)]],
    ["和平街十二区10号楼", [(116.42537,39.968319), (116.425788,39.968323), (116.425793,39.968233), (116.42537,39.968221)]],
    ["和平街十二区8号楼", [(116.425364,39.967999), (116.425793,39.968007), (116.425777,39.967925), (116.425364,39.967916)]],
    ["和平街十二区16号楼", [(116.425906,39.96806), (116.426121,39.968069), (116.426121,39.967974), (116.426314,39.967978), (116.426308,39.968311), (116.426389,39.968307), (116.42641,39.967892), (116.426131,39.967879), (116.426121,39.967925), (116.425906,39.967921)]],
    ["和平街十二区7号楼", [(116.425219,39.967736), (116.425756,39.96776), (116.425751,39.967694), (116.425327,39.967682), (116.425316,39.967534), (116.425225,39.96753)]],
    ["和平街十二区12号楼", [(116.42596,39.967707), (116.426367,39.967715), (116.426378,39.967596), (116.426432,39.967592), (116.426437,39.967411), (116.426249,39.967415), (116.426244,39.967596), (116.425965,39.967596)]],
    ["和平街十二区5号楼", [(116.425219,39.967279), (116.425316,39.967279), (116.425316,39.96737), (116.425413,39.967361), (116.425429,39.96686), (116.425236,39.966852)]],
    ["和平街十二区15号楼", [(116.425847,39.967279), (116.426421,39.967296), (116.426426,39.967205), (116.425852,39.967197)]],
    ["和平街十二区6号楼", [(116.425504,39.967016), (116.425869,39.967016), (116.425847,39.966942), (116.425498,39.966934)]],
    ["和平街十二区14号楼", [(116.425922,39.967024), (116.426287,39.967032), (116.426282,39.966963), (116.425912,39.96695)]],
    ["和平街十二区4号楼", [(116.425209,39.96665), (116.425305,39.966646), (116.425316,39.966514), (116.425761,39.966519), (116.425761,39.966453), (116.425225,39.966445)]],
    ["和平街十二区13号楼", [(116.425815,39.966794), (116.425922,39.966794), (116.425928,39.966547), (116.42633,39.966551), (116.42633,39.966802), (116.426426,39.966806), (116.426437,39.966469), (116.42582,39.966457)]],
    ["和平街十二区3号楼", [(116.425171,39.966235), (116.425777,39.966255), (116.425799,39.966058), (116.42567,39.966054), (116.425665,39.966157), (116.425295,39.966157), (116.425295,39.966042), (116.425177,39.966038)]],
    
    ["和平街十二区甲1号楼", [(116.425214,39.965947), (116.425338,39.965943), (116.425343,39.965495), (116.425225,39.965495)]],
    ["和平街十二区甲3号楼", [(116.425498,39.965918), (116.425729,39.965922), (116.42574,39.965844), (116.425692,39.965844), (116.425692,39.965774), (116.425541,39.965778), (116.425541,39.965844), (116.425493,39.965844)]],
    ["和平街十二区甲2号楼", [(116.425498,39.965639), (116.425729,39.965639), (116.425729,39.965552), (116.425681,39.965552), (116.425681,39.965491), (116.425531,39.965482), (116.425531,39.965544), (116.425498,39.965569), (116.425504,39.965614)]],
    ["和平街十二区1号楼", [(116.425252,39.965392), (116.425375,39.965388), (116.42538,39.965306), (116.425745,39.965306), (116.425751,39.9654), (116.425852,39.965396), (116.425863,39.965191), (116.425262,39.965195)]],
    ["和平街十三区6号楼", [(116.425257,39.964997), (116.425863,39.964997), (116.425869,39.964681), (116.425756,39.964681), (116.425751,39.964915), (116.425364,39.964911), (116.42537,39.964668), (116.425273,39.964664)]],
    ["和平街十三区17号楼", [(116.42603,39.964993), (116.426652,39.965001), (116.426652,39.964911), (116.42603,39.964899)]],
    ["和平街十三区5号楼", [(116.425777,39.964549), (116.425879,39.964545), (116.425885,39.964216), (116.425788,39.964216)]],
    
    ["化研院南小区5号楼", [(116.426797,39.965632), (116.427902,39.965689), (116.427896,39.965525), (116.426802,39.965517)]],
    ["和平街十二区22号楼", [(116.428047,39.965718), (116.429071,39.965747), (116.429071,39.965533), (116.428052,39.965521)]],
    ["和平街十二区23号楼", [(116.429221,39.965722), (116.430139,39.965722), (116.430144,39.965558), (116.429216,39.965541)]],
    ["和平街十二区25号楼", [(116.429152,39.966121), (116.430117,39.966138), (116.430123,39.96594), (116.429152,39.96592)]],

    ["和平街十五区2号楼", [(116.430235,39.966409), (116.43031,39.966417), (116.430305,39.966347), (116.430589,39.966343), (116.430589,39.96638), (116.430691,39.966368), (116.430691,39.966277), (116.430654,39.966232), (116.430273,39.966228), (116.430267,39.966277), (116.430219,39.966273)]],
    ["和平街十五区4号楼", [(116.430267,39.965776), (116.430343,39.965759), (116.430621,39.965722), (116.430729,39.965759), (116.430729,39.965661), (116.430729,39.965759), (116.430729,39.965661), (116.430686,39.965648), (116.430686,39.965603), (116.430305,39.965599), (116.430316,39.965652), (116.430257,39.965665)]],
    ["和平街十五区1号楼", [(116.430836,39.966684), (116.430943,39.966651), (116.430949,39.966623), (116.431147,39.966614), (116.431142,39.966651), (116.431228,39.966676), (116.431228,39.966565), (116.431174,39.966573), (116.431185,39.966512), (116.430895,39.966503), (116.4309,39.966553), (116.430847,39.966553)]],


]   

buildings = []
for i, (name, points) in enumerate(bds):
    if points[0] != points[-1]:
        points += [points[0]]
    points = [gcj2wgs(*p[::-1])[::-1] for p in points]
    buildings.append({
        "id": i,
        "name": name,
        "points": points,
    })

In [42]:
G_folium = folium.Map(
    location=[lat_cen, lon_cen],
    control_scale=True
)
for bd in buildings:
    folium.PolyLine(
        locations=[p[::-1] for p in bd["points"]],
        opacity=0.8,
        weight=3,
        popup=bd["name"],
    ).add_to(G_folium)
G_folium.save("figure/test.html")